<a href="https://colab.research.google.com/github/Foluwa/30-days-of-udacity-computer-vision/blob/master/Detectron2_Number_Plates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and Inference with Detectron 2 Framework for Number Plates Detection
<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

## Access your drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/My\ Drive

In [0]:
mkdir NumberPlates

In [0]:
cd NumberPlates

## Install and build required dependencies


In [0]:
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

### And restart the Runtime

## Clone into Gdown.pl and download the dataset

In [0]:
!git clone https://github.com/circulosmeos/gdown.pl

In [0]:
cd gdown.pl

In [0]:
!sudo chmod 777 gdown.pl

In [0]:
!./gdown.pl https://drive.google.com/file/d/1u1VNPrDPP6AePoiYESldTBepFaamrMbY plates_coco.zip
!mv plates_coco.zip ../

In [0]:
!cd ../
!unzip -q plates_coco.zip

## Get pretrained model in case you want to do inference straight away.

In [0]:
!mkdir output
!cd gdown.pl
!./gdown.pl https://drive.google.com/file/d/1hMCczENeCLd-jq7KGW9ak8B-QkWVUoXg ../output/model_final.pth

## Training

In [0]:
# basic libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# detectron2 utilities
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances

# register dataset
register_coco_instances("plates", {}, "./plates_coco/annotations.json", "./plates_coco/")

plates_metadata = MetadataCatalog.get("plates")
dataset_dicts = DatasetCatalog.get("plates")

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import torch, os

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml")


cfg.DATASETS.TRAIN = ("plates",)
cfg.DATASETS.TEST = ()   
cfg.DATALOADER.NUM_WORKERS = 2

# initialize model from model zoo
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"  
#cfg.MODEL.WEIGHTS = 'output/model_final.pth' 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.SOLVER.BASE_LR = 0.00025

cfg.SOLVER.MAX_ITER = 10000    
cfg.SOLVER.CHECKPOINT_PERIOD = 500

# we've only one class - plate
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Inference

In [0]:
import random as r
import math
cfg = get_cfg()

cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml")

# detector threshold

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  
cfg.MODEL.WEIGHTS = 'output/model_final.pth'
cfg.DATASETS.TEST = ("plates", )
predictor = DefaultPredictor(cfg)

# get images with glob function to filelist to iterate through. Use any directory for testing, with your prefered format of 
# images: .png, .jpg, .jpeg.

filelist = glob.glob('*.jpg')

for i in range(10):
    img = cv2.imread(filelist[i])
    
    # prediction
    outputs = predictor(img)
    
    ### uncomment the below script in case you want to see the detector visualisations
    
    ### v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
    ### v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    ### cv2_imshow(v.get_image()[:, :, ::-1])
    
    # getting prediction bboxes from model outputs
    
    boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()[0]
    x2 = math.ceil(boxes[0])
    x1 = math.ceil(boxes[1])
    y2 = math.ceil(boxes[2])
    y1 = math.ceil(boxes[3])
    crop_img = img[x1:y1,x2:y2]
    #crop_img = cv2.resize(crop_img, (500,250))
    
    # showing original image
    cv2_imshow(img)
    
    # showing cropped number plate
    cv2_imshow(crop_img)